# Task
Create a program to read the file "/content/amenity_analysis_results.csv", filter out the 1% most expensive houses, one-hot encode the 'Appliance' column and all other string columns, train an XGBoost model to predict the 'Price' column, cross-evaluate the hyperparameters, and then show a list of the highest errors between predicted and actual rent.

## Load data

### Subtask:
Load the data from the specified CSV file into a pandas DataFrame.


**Reasoning**:
Import pandas and load the CSV file into a DataFrame, then display the head and info.



In [ ]:
import pandas as pd
import numpy as np
import sklearn
df = pd.read_csv("/content/sample_data/floridaHomeDetailsV3.csv",on_bad_lines='skip')

df.head().to_csv("floridaHomeDetailsV3UnEncoded.csv")

# Function to extract zipcode from URL
def extract_zipcode_from_url(url):
    if isinstance(url, str):
        try:
            parts = url.split('/homedetails/')[1].split('/')
            # Assuming the zipcode is the part before the _zpid
            zip_part = parts[-2] if '-' in parts[-2] else parts[-1]
            zipcode = zip_part.split('-')[0]
            return zipcode
        except:
            return np.nan
    return np.nan

# Function to recalculate Average Rental Price
def recalculate_average_rent(row):
    times_rented = row['Times Rented']
    last_rental_price = row['Last Rental Price']
    average_time_on_market = row['Average Time on Market'] # Assuming this is the old average

    # Check for conditions where calculation is not possible or would result in NaN
    if pd.isna(times_rented) or times_rented < 2 or pd.isna(last_rental_price) or pd.isna(average_time_on_market):
        return np.nan
    else:
        # Apply the formula: (Old Average * Times Rented - Last Rental Price) / (Times Rented - 1)
        # Need to be careful if Times Rented is 1, as it would result in division by zero.
        if times_rented - 1 == 0:
            return np.nan # Avoid division by zero
        else:
            new_average = (average_time_on_market * times_rented - last_rental_price) / (times_rented - 1)
            return new_average

# Apply the function to fill missing zipcodes
df['Zipcode'] = df.apply(
    lambda row: extract_zipcode_from_url(row['URL']) if pd.isna(row['Zipcode']) else row['Zipcode'],
    axis=1
)

# Apply the function to recalculate Average Time on Market
df['Average Time on Market'] = df.apply(recalculate_average_rent, axis=1)

# Remove the top 10% of outliers based on 'Most Recent Time on Market'
cutoff = df['Price'].quantile(0.95)
print(f"Using a cutoff of {cutoff:.2f} ")

# Create a new DataFrame without the top 10% of outliers
df = df[df['Price'] <= cutoff]

display(df.head())
display(df.info())
display(df.columns)
df['Price'].describe()

In [ ]:
# Calculate 'Rent Estimate to actual Price' column
df['Rent Estimate to actual Price'] = df['Rent Zestimate'] / df['Price']

# Calculate 'price to area average by zipcode' column
# First, calculate the average price per zipcode
average_price_by_zipcode = df.groupby('Zipcode')['Price'].transform('mean')

# Then, calculate the ratio of 'Price' to the average price by zipcode
df['price to area average by zipcode'] = df['Price'] / average_price_by_zipcode

# Function to extract the numerical month of the most recent 'Listed for rent' or 'Sold' event
import json

def get_most_recent_month(events_string):
    try:
        events = json.loads(events_string)
        # Filter for 'Listed for rent' or 'Sold' events and sort by date
        relevant_events = sorted([e for e in events if e['type'] in ['Listed for rent', 'Sold']],
                                 key=lambda x: x['date'], reverse=True)
        if relevant_events:
            # Get the date of the most recent relevant event
            most_recent_date_str = relevant_events[0]['date']
            # Extract and return the numerical month
            return pd.to_datetime(most_recent_date_str).month
    except:
        # Return NaN if there's an error parsing or no relevant events
        return np.nan
    return np.nan

# Apply the function to create the 'month listed' column
df['month listed'] = df['Events'].apply(get_most_recent_month)

display(df[['Rent Estimate to actual Price', 'price to area average by zipcode', 'month listed']].head())

## Filter Data

### Subtask:
Filter out the 1% most expensive houses based on the 'Price' column.

**Reasoning**:
Calculate the 99th percentile of the 'Price' column and filter the DataFrame to exclude houses with prices above this threshold. Display the head and info of the filtered DataFrame.

In [ ]:

# Function to split and one-hot encode the 'Appliances' column
def encode_appliances(df):
    # Split the 'Appliances' string into a list of appliances, handling NaN values
    appliances_list = df['Appliances'].str.split(', ').apply(lambda x: x if isinstance(x, list) else [])
    # Get all unique appliances
    all_appliances = sorted(list(set([item for sublist in appliances_list for item in sublist])))
    # Create new columns for each appliance and fill with 1 if the appliance is present
    for appliance in all_appliances:
        df[f'Appliance_{appliance}'] = appliances_list.apply(lambda x: 1 if appliance in x else 0)
    return df

# Apply one-hot encoding to 'Appliances'
df_filtered = encode_appliances(df)

# Apply one-hot encoding to other string columns
df_filtered = pd.get_dummies(df_filtered, columns=['Cooling', 'Heating', 'Parking', 'Laundry', 'Home Type'])

display(df_filtered.head())
display(df_filtered.info())

In [ ]:
import pandas as pd
import numpy as np

def fill_nan_with_median(df):
    """
    Fills NaN values in a DataFrame with the median of each column using a manual loop.
    If the median cannot be calculated for a column, the NaN values in that column will remain NaN.
    """
    df_copy = df.copy()
    for col in df_copy.columns:
        try:
            # Attempt to calculate the median. Use try-except to catch errors for non-numeric columns
            median_val = df_copy[col].median()
            if not pd.isna(median_val):
                df_copy[col] = df_copy[col].fillna(median_val)
        except:
            # If median cannot be calculated (e.g., non-numeric column), leave NaN values as they are
            pass # Do nothing, leave NaN values
    return df_copy


# Apply the custom NaN filling function
df_filled = fill_nan_with_median(df_filtered.copy())

# Shuffle the DataFrame
df_shuffled = df_filled.sample(frac=1, random_state=41).reset_index(drop=True)

display(df_shuffled.head())
display(df_shuffled.info())

In [ ]:
df_shuffled.head().to_csv("floridaHomeDataIncomeEncodedV2.csv" ,index=False)

## Train XGBoost Model

### Subtask:
Train an XGBoost model to predict the 'Price' column.

**Reasoning**:
Separate the features (X) and target variable (y), split the data into training and testing sets, and train an XGBoost Regressor model.

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np

# Separate features (X) and target variable (y)
# Exclude non-numeric and irrelevant columns
X = df_shuffled.drop(columns=['Price', 'Street Address', 'City', 'State', 'URL', 'Appliances', "Image URLs", "Date Details Fetched", "Events","Rent Estimate to actual Price","price to area average by zipcode"])
y = df_shuffled['Price']

df_shuffled.to_csv("testEncoded.csv")

# Select only numeric columns for training
X = X.select_dtypes(include=np.number)

# Replace infinite values with NaN and then fill NaN with the median
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.fillna(X.median())

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_test.head().to_csv("testOutput.csv")

# Initialize and train the XGBoost Regressor model
xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.01, random_state=42)
xgbr.fit(X_train, y_train)

print("XGBoost model training complete.")

## Cross-evaluate Hyperparameters and Evaluate Model

### Subtask:
Cross-evaluate the hyperparameters of the trained XGBoost model and calculate the R2 score.

**Reasoning**:
Use cross-validation to evaluate the model's performance across different subsets of the data and calculate the R2 score on the test set to assess the model's goodness of fit.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error

# Cross-evaluate the model using cross-validation
# Using R2 as the scoring metric
scores = cross_val_score(xgbr, X_train, y_train, cv=10, scoring='r2')

print(f"Cross-validation R2 scores: {scores}")
print(f"Mean cross-validation R2 score: {scores.mean()}")

# Predict on the test set
y_pred = xgbr.predict(X_test)

# Calculate the R2 score on the test set
r2 = r2_score(y_test, y_pred)

print(f"R2 score on the test set: {r2}")

scores = cross_val_score(xgbr, X_train, y_train, cv=10, scoring='neg_mean_absolute_error')

print(f"Cross-validation MAE scores: {scores}")
print(f"Mean cross-validation MAE score: {scores.mean()}")



## Analyze Highest Errors

### Subtask:
Show a list of the highest errors between predicted and actual rent.

**Reasoning**:
Calculate the absolute errors between the predicted and actual prices, sort the results by error in descending order, and display the instances with the highest errors.

In [ ]:
# Calculate the absolute errors
errors = abs(y_test - y_pred)

# Create a DataFrame to show actual price, predicted price, and error
error_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Error': errors})

# Get the original index from the test set
error_df = error_df.join(df_shuffled['URL'], how='left')


# Sort by error in descending order and display the top errors
display(error_df.sort_values(by='Error', ascending=False).head())
error_df.sort_values(by='Error', ascending=False).to_csv('error_df.csv')

In [ ]:
%pip install matplotlib

## SHAP Analysis

### Subtask:
Generate a SHAP summary plot to visualize feature importance.

**Reasoning**:
Use the `shap` library to calculate SHAP values for the test set and generate a summary plot to show the impact of each feature on the model's predictions.

In [ ]:
import shap

# Create a SHAP explainer object
explainer = shap.TreeExplainer(xgbr)

# Calculate SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Generate the SHAP summary plot
shap.summary_plot(shap_values, X_test)

print(df_shuffled['Price'].describe())

## Calculate MAE, MSE, and RMSE

### Subtask:
Calculate and display the Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE) of the model's predictions in one cell.

**Reasoning**:
Use `sklearn.metrics.mean_absolute_error`, `sklearn.metrics.mean_squared_error`, and `numpy.sqrt` to calculate MAE, MSE, and RMSE using the actual and predicted values and display the results.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Select only numeric columns from the shuffled DataFrame
df_shuffled_numeric = df_shuffled.select_dtypes(include=np.number)

# Calculate the correlation matrix
correlation_matrix_shuffled = df_shuffled_numeric.corr()

# Set the figure size
plt.figure(figsize=(18, 16))

# Create the correlation heatmap
sns.heatmap(correlation_matrix_shuffled, cmap='coolwarm', annot=False)
plt.title('Correlation Heatmap of Numeric Features (Shuffled Data)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred)
plt.title('Actual vs. Predicted Price')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.show()